# Métrica 1.1.09: "Validar estructura de CURP (pareja)"

In [1]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
import traceback
# Asumimos que tienes un archivo config.py con tus variables de conexión
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_09_CURP_PAREJA_ESTRUCTURA"
SOURCE_COLLECTION_NAME = "all_data_20251021"

# --- 2. LÓGICA PURA DE LA MÉTRICA (Reutilizada de 1.1.01) ---

def validar_curp_estructura(curp: str) -> bool:
    """
    Valida la estructura de una CURP según la métrica 1.1.01/1.1.09,
    incluyendo la validación de las 32 entidades federativas.
    """
    if not isinstance(curp, str):
        return False
        
    # Esta es la expresión regular que implementa TODOS tus puntos.
    curp_regex = (
        "^[A-Z][AEIOU][A-Z]{2}"  # 4 letras, 2da es vocal
        "[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|[12][0-9]|3[01])"  # Fecha aammdd
        "[HM]"  # Sexo H o M
        # Abreviación de Entidad Federativa (32 estados + Nacido Extranjero)
        "(AS|BC|BS|CC|CS|CH|CL|CM|DF|DG|GT|GR|HG|JC|MC|MN|MS|NT|NL|OC|PL|QT|QR|SP|SL|SR|TC|TS|TL|VZ|YN|ZS|NE)"
        "[B-DF-HJ-NP-TV-Z]{3}"  # 3 Consonantes
        "[A-Z0-9]"  # Alfanumérico (Homoclave)
        "[0-9]$"  # Numérico (Dígito verificador)
    )
    
    if re.match(curp_regex, curp):
        return True
    else:
        return False

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_curp_pareja():
    """
    Procesa TODAS las declaraciones para la Métrica 1.1.09
    """
    client = None
    # Contadores
    total_documentos = 0
    cumple = 0
    no_cumple = 0
    sin_dato = 0
    na = 0 # No Aplica (para quienes no tienen pareja)
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"\nConectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"--- INICIANDO PROCESAMIENTO (Métrica 1.1.09 - Estructura CURP Pareja) ---")

        # Proyección optimizada para traer el 'id' y la sección 'datosPareja'
        proyeccion = {
            "_id": 1,
            "id": 1,
            "declaracion.situacionPatrimonial.datosPareja": 1 
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]
            string_id = doc.get("id", "N/D")
            
            curp_pareja = None
            resultado_metrica = "SIN_DATO" # Valor por defecto
            
            try:
                # Navegación segura
                datos_pareja = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosPareja")

                # --- Lógica de la Métrica ---
                if not isinstance(datos_pareja, dict):
                    # No existe la sección 'datosPareja'
                    resultado_metrica = "SIN_DATO"
                
                elif datos_pareja.get("ninguno") is True:
                    # El declarante marcó "Ninguno" en la sección de pareja
                    resultado_metrica = "N/A"
                
                else:
                    # El declarante SÍ tiene pareja, buscamos el CURP
                    curp_pareja = datos_pareja.get("curp")
                    
                    if curp_pareja and curp_pareja.strip() != "":
                        if validar_curp_estructura(curp_pareja):
                            resultado_metrica = "CUMPLE"
                        else:
                            resultado_metrica = "NO_CUMPLE"
                    else:
                        # Tiene pareja pero no proporcionó el CURP
                        resultado_metrica = "SIN_DATO"

            except AttributeError:
                resultado_metrica = "SIN_DATO" # Ocurrió un error leyendo la estructura
            
            # Actualizamos contadores
            if resultado_metrica == "CUMPLE": cumple += 1
            elif resultado_metrica == "NO_CUMPLE": no_cumple += 1
            elif resultado_metrica == "N/A": na += 1
            else: sin_dato += 1 # SIN_DATO

            # --- CONSERVAMOS TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id} (id: {string_id})")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > CURP Pareja: '{curp_pareja}'")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")

            # --- E. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            filtro = { "_id": original_id }
            actualizacion = { "$set": { METRIC_ID: resultado_metrica } }
            target_collection.update_one(filtro, actualizacion, upsert=True)
            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.09):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {no_cumple}")
        print(f"  > Métrica 'N/A' (Sin pareja declarada): {na}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        traceback.print_exc()
    finally:
        if client: client.close(); print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones_curp_pareja()


Conectando a MongoDB en sistema1...
¡Conexión exitosa!
--- INICIANDO PROCESAMIENTO (Métrica 1.1.09 - Estructura CURP Pareja) ---

Procesando Documento ID: 68f81b8800535f910a29f694 (id: 194916)
--- RESULTADO DEL ANÁLISIS ---
  > CURP Pareja: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695 (id: 59837)
--- RESULTADO DEL ANÁLISIS ---
  > CURP Pareja: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696 (id: 31074)
--- RESULTADO DEL ANÁLISIS ---
  > CURP Pareja: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f696!

Procesando Documento ID: 68f81b8800535f910a29f

KeyboardInterrupt: 